In [1]:
import rebound
import sys
#change the next line to reflect where you have downloaded the source code
sys.path.insert(0, '/Users/kvolk/Documents/GitHub/SBDynT/src')
import sbdynt as sbd


# *Tools to directly set up a rebound integration from JPL orbit fits*
These routines use JPL's small body database to pull best-fit orbits and (if you want clones) an orbit-fit  covariance matrix for a small body given its designation and initializes a rebound simulation instance with that object and the sun and planets for the epoch of the orbit fit 

see detailed-examples-of-JPL-and-Horizons-query-functions.ipynb for details on the JPL queries

## *Example of setting up an integration for a single object*

Note that the rebound simulation will have the following units: <br /> 
distance:  au <br /> 
time: years (1 year==365.25 days) <br /> 
mass: solar masses <br /> 

The designation of the small body can be given in most common formats, here we use packed provisional format, but other formats for this example object would include <br /> 
small_body= '2014XT40' (unpacked provisional des without a space) <br /> 
small_body= '2014 XT40' (unpacked provisional des with a space)  <br /> 
small_body = '535167' (this object is also numbered, so this is the number) <br /> 
small_body = 'r5167' (that number can be packed or not) <br /> 

The most basic use of the sbd.**initialize_simulation** function will return a simulation with all the major planets initialized at the epoch of the small body's best-fit orbit:

In [2]:
small_body = 'K14X40T'
flag, epoch, sim = sbd.initialize_simulation(des=small_body)

if(flag):
    print("Sucessfully added ", small_body, "to a simulation at epoch ", epoch)
    print("the best fit clone has a heliocentric orbit of:")
    
    helio_orbit = sim.particles[small_body].orbit(sim.particles[0])
    print("a=",helio_orbit.a)
    print("e=",helio_orbit.e)
    print("i=",helio_orbit.inc)
    print("node=",helio_orbit.Omega)
    print("aperi=",helio_orbit.omega)
    print("mean anomaly=",helio_orbit.M)

    print("\n\nThe status of the rebound simulation instance generated is:")
    sim.status(showAllFields=False)

Sucessfully added  K14X40T to a simulation at epoch  2457293.5
the best fit clone has a heliocentric orbit of:
a= 39.37435526600913
e= 0.19030770354867474
i= 0.12811604691522185
node= -2.3764470316954385
aperi= 3.0439841132466654
mean anomaly= 0.15561466604368324


The status of the rebound simulation instance generated is:
---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	Jul 12 2024 21:06:23
Number of particles: 	10
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.012000
---------------------------------
<rebound.particle.Particle object at 0x159fa3bd0, m=1.0 x=0.0036117142367445344 y=0.0007650357929431655 z=-0.00015742679050831225 vx=0.0006316324676849848 vy=0.0023702294754373034 vz=-1.867735571010212e-05>
<rebound.particle.Particle object at 0x159fa31d0, m=1.660120825450693e-07 x=0.36211838598210727 y=-0.034105571075160136 z=-0.035897753115829996 vx=-0.9912023307429476 vy=10.690165545363518 vz=0.96427

**Specifying planets, adding clones, saving logs and initial conditions**

You can also specify the planets you want to include in the simulation (for example, the terrestrial planets aren't really needed if you are integrating a TNO!). This is done by passing a list of **planets** in the function call. Note that as a shortcut, you can also use planets=['outer'] to use just the giant planets. During the initialization, the masses of planets not included in the call are accounted for and added to the Sun, and the timestep of the simulation is set to a value reasonable for the closest-in included planet. **\*Note\*** that if you are simulating very close-in comets or near earth asteroids (closer-in than Mercury's orbit), you may need to reset the default timestep to an smaller value to ensure you are accurately resolving their perihelion passages!

If clones to sample the orbit uncertainy are desired, then the JPL orbit fit covariance matrix will be used to generate
them (the best-fit orbit will always be the first particle and then clones are added in addition to the best-fit orbit). Cloning will happen if **clones** > 0 in the sbd.initialize_simulation function call. If the clones parameter is not included in the function call, it is assumed that clones=0.

You can also choose to save the queried covariance matrix that was used for the cloning to a pickle file (useful if you run the same object in the future and want to compare a new orbit-fit to an older one). That is achieved using the **save_sbdb** parameter, which can be set to True (saves to a default file name of \<designation\>-\<date\>-.pkl) or to a string with your desired file name. The default location for where files will be saved is the directory you are currently in, but can also be specified using **datadir** (as is done in the example below).

Setting the **saveic** parameter to True will save the initialized rebound simulation instance to a simulation archive binary file with the name \<designation\>-ic.bin. You can also specify a string with a different file name. If the file already exists, the initial conditions will be appended.


Finally, you can specify whether you want to generate a log, either to screen or to a file. If **logfile**=True, log messages will be appended to \<designation\>-log.txt (in either the current directory of the datadir); logfile can also be set to a string with a desired file name for the log or set to 'screen' to print the log messages to the screen instead of a file.


In [3]:
small_body = 'K14X40T'
n_clones = 5
flag, epoch, sim = sbd.initialize_simulation(des=small_body,
                                             planets=['Jupiter','Saturn','Uranus','Neptune'],
                                             clones=n_clones,
                                             save_sbdb=True,
                                             saveic=True,
                                             datadir = 'outputs-from-example-notebooks',
                                             logfile='screen')

#print out the simulation status
if(flag):
    print("Sucessfully added ", small_body, "to a simulation at epoch ", epoch)
    sim.status(showAllFields=False)


SBDB query results saved to outputs-from-example-notebooks/K14X40T-Sep-19-2024.pkl

simulation epoch: 2457293.5

Rebound simulation initial conditions saved to outputs-from-example-notebooks/K14X40T-ic.bin

Sucessfully added  K14X40T to a simulation at epoch  2457293.5
---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	Jul 12 2024 21:06:23
Number of particles: 	11
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.400000
---------------------------------
<rebound.particle.Particle object at 0x159fa2350, m=1.0000059769986274 x=0.003615969000500503 y=0.000766569114419985 z=-0.00015749063777582636 vx=0.0006191546623273798 vy=0.002405415036719666 vz=-1.7756306065976055e-05>
<rebound.particle.Particle object at 0x159fa27d0, m=0.0009547919099366768 x=-4.910099980978657 y=2.227725678438094 z=0.10054518793237059 vx=-1.1710501125842254 vy=-2.3801686961254913 vz=0.03609390700047327>
<rebound.particle.Particle object a

You can proceed from here with regular rebound commands, but we have also included some tools
to directly run the rebound simulations, writing outputs to a simulation archive binary file. 

The simulation defaults to using rebound's mercurius integrator with for the timestep as set above based on the planets in the simulation. **Note of caution:** the simulation
does not check to see if a small body gets closer to the sun than the innermost included planet!!! This
means that the integration timestep might become unreasonable if the small body evolves too far inward.
(Or you could be missing important perturbers, such as the terrestrial planets, if you initialized the 
simulation with only the outer planets! Check for such conditions when analyzing the output)


In the simple example below of sbd.**run_simulation**, the simulation will be advanced using the default timestep to time **tmax** (in years), saving the state of the simulation every **tout** years. A default name will be used for the simulation archive file, as printed to screen.

In [4]:
tmax = 1e6
tout = 1e3

flag, sim = sbd.run_simulation(sim, des=small_body, 
                               tmax=tmax, tout=tout,
                               datadir = 'outputs-from-example-notebooks',
                               deletefile=True,
                               logfile='screen')


Running K14X40T from 0.0 to 1000000.0 years 
using mercurius outputting every 1000.0 years 
to simulation archivefile outputs-from-example-notebooks/K14X40T-simarchive.bin
starting at 2024-09-19 16:39:25.139443

finishing at 2024-09-19 16:39:28.090583



There are some additional parameters you can use in sbd.**run_simulation**

In the example below, we will re-initialize a rebound simulation instance from the saved initial conditions file using sbd.**initialize_simulation_from_simarchive**. (If **archivefile** is omitted from this function call, the simulation would be initialized from the default file saved in the example above and the simulation time would be 1e6 years.)

We then run that simulation with sbd.**run_simulation**, but specifying the use of ias15 rather than mercurius with the **integrator** parameter. We will also save to a different filename than the default (**archivefile** parameter) and tell sbd.**run_simulation** to delete any pre-existing simulation archive of that name using the **deletefile** parameter. 

**Note** ias15 is much slower than mercurius! We will shorten the integration length by a factor of 100 to keep the example running quickly



In [5]:
flag, sim, n_clones = sbd.initialize_simulation_from_simarchive(des=small_body,
                                                                archivefile = 'K14X40T-ic.bin',
                                                                datadir = 'outputs-from-example-notebooks',
                                                                logfile='screen')        


tmax = 1e4
tout=5e2
flag, sim = sbd.run_simulation(sim, des=small_body, 
                               tmax=tmax, tout=tout,
                               integrator = 'ias15',
                               archivefile = 'custom-new-file-name.bin',
                               datadir = 'outputs-from-example-notebooks',
                               deletefile=True,
                               logfile='screen')


Loaded integration for K14X40T from outputs-from-example-notebooks/K14X40T-ic.bin
simulation is at time 0.0 years
integrator is ias15

Found K14X40T and 5 clones in the simulation

Running K14X40T from 0.0 to 10000.0 years 
using ias15 outputting every 500.0 years 
to simulation archivefile outputs-from-example-notebooks/custom-new-file-name.bin
starting at 2024-09-19 16:39:28.093904

finishing at 2024-09-19 16:39:28.441208



## *Example of setting up an integration with a list of small bodies*

**No clones allowed** in this instance as the cloning procedure relies on orbit fits that have independent epochs!

Also note that none of our analysis routines are optimized for this (they all assume a typical use-case of one small body plus clones per simulation), so this funcitonality is really only to take advantage of the built-in 



In [6]:
# as above, designations can be packed, unpacked, numbers, comets, etc
list_of_small_bodies = ['K14X40T','2016 SW50', '15760','29P','179P/Jedicke']

# initialize the simulation without specifying planets (it will thus include all the major planets)
flag, epoch, sim = sbd.initialize_simulation_at_epoch(des=list_of_small_bodies, epoch=2457019.)

#print out the simulation status
if(flag):
    sim.status(showAllFields=False)


---------------------------------
REBOUND version:     	4.4.1
REBOUND built on:    	Jul 12 2024 21:06:23
Number of particles: 	14
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.012000
---------------------------------
<rebound.particle.Particle object at 0x159fa1550, m=1.0 x=0.002823131996485608 y=-0.0008788892248169295 z=-0.00013677120342035865 vx=0.0014620955720366165 vy=0.0019026198829254328 vz=-3.566762704994574e-05>
<rebound.particle.Particle object at 0x159fa3c50, m=1.660120825450693e-07 x=0.2850152192904971 y=-0.3078755512113022 z=-0.051110918170975725 vx=5.526647147789773 vy=7.446690400192037 vz=0.10135077000098804>
<rebound.particle.Particle object at 0x159fa1550, m=2.447838287784771e-06 x=0.4893659122545278 y=-0.5415398110768662 z=-0.0356247413395807 vx=5.444332334171796 vy=4.917084511306496 vz=-0.24677331732671584>
<rebound.particle.Particle object at 0x159fa3c50, m=3.0404326489475004e-06 x=-0.09036731852085139 y=0.9781477163